# Copy selected stereo images, rename them and extract patch

In [1]:
###import libraries
import csv
import shutil
from pathlib2 import Path

In [2]:
###GLOBAL VARIABLES
#paths to input and output csv files
str_refFile = '../files/csv/biograd_calm_gestures.csv'
str_outFile = '../files/csv/biograd-A.csv'
str_imgsPath = '/home/arturokkboss33/Databases/underwater_gestures/biograd-A/stereo-imgs/'

#string names to rename the files
str_location = 'biograd-A'
str_underscore = '_'
str_side_left = 'left'
str_side_right = 'right'

In [3]:
###CODE
stereo_pairs_list = list()
imgs_nums_dict = {}

#go through stereo file, and create a list of the images and associated index
with open(str_refFile,'r') as stereofile:
    imageReader = csv.reader(stereofile, delimiter=',')
    flag_skip_row = True
    for row in imageReader:
        if not flag_skip_row:
            #Extract information about the stereo imgs from the csv file
            path_left = Path(row[0])
            str_left = path_left.stem
            str_img_num = int(str_left.split('_')[2])
            formated_num = '{:05d}'.format(str_img_num)
            stereo_pairs_list.append(formated_num)     
            
        else:
            flag_skip_row = False

#sort the list, and create a hash table/dictionary with the correct order
stereo_pairs_list.sort()
for idx, element in enumerate(stereo_pairs_list):
    imgs_nums_dict[element] = idx
    
#print(imgs_nums_dict)

In [4]:
###CODE
flag_copy_images = True
finalFile_fieldNames = ['stereo left','stereo right','selected img','label name', 'label id', 'roi params']

#open output file to write when necessary
ofile = open(str_outFile,'w')
writer_finalFile = csv.DictWriter(ofile,fieldnames=finalFile_fieldNames)
writer_finalFile.writeheader()

#go through stereo file, and copy imgs to new location and create final csv file
with open(str_refFile,'r') as stereofile:
    imageReader = csv.reader(stereofile, delimiter=',')
    flag_skip_row = True
    for row in imageReader:
        if not flag_skip_row:
            #Extract information about the stereo imgs from the csv file
            path_left = Path(row[0])
            path_right = Path(row[1])
            str_left = path_left.stem
            str_right = path_right.stem
            str_extension = path_right.suffix
            img_num = int(str_left.split('_')[2])
            
            #Prepare new image names and location to copy them
            formated_raw_num = '{:05d}'.format(img_num)
            formated_true_num = '{:05d}'.format(imgs_nums_dict[formated_raw_num])
            final_str_left = "%s%s%s%s%s%s"%(str_location,str_underscore,formated_true_num,str_underscore,
                                             str_side_left,str_extension)
            final_str_right = "%s%s%s%s%s%s"%(str_location,str_underscore,formated_true_num,str_underscore,
                                             str_side_right,str_extension)
            
            #Copy images to new location with new image name
            dst_left = "%s%s"%(str_imgsPath,final_str_left)
            dst_right = "%s%s"%(str_imgsPath,final_str_right)
            if flag_copy_images:
                shutil.copy(str(path_left),dst_left)
                shutil.copy(str(path_right),dst_right)
                
            #Create new csv file
            writer_finalFile.writerow({'stereo left':dst_left,
                                      'stereo right':dst_right,
                                      'selected img':row[2],
                                      'label name':row[3],
                                      'label id':row[4],
                                      'roi params':row[5]}) 
            
            
        else:
            flag_skip_row = False

ofile.close()